# Welcome to Lucien Notebook! 🎉

Welcome to Lucien's interactive programming environment! Here you can:
- Write and execute code
- Create data visualizations
- Explore and create interesting content with Lucien Agent

## How to Reference Files in Your Working Directory 📁

In this notebook, you have two ways to import files from your working directory:

### 1. Using Absolute Paths (Recommended)

```python
# Example
with open('/absolute/path/to/your/file.txt', 'r') as f:
    content = f.read()
```

### 2. Using Relative Paths After Changing Directory

```python
import os

# First change to your working directory
os.chdir('/path/to/your/working/directory')

# Then use relative paths
with open('file.txt', 'r') as f:
    content = f.read()
```

> ⚠️ **Note**: It is **not recommended** to use relative paths directly in notebooks, as the base directory might not be what you expect.

Start exploring now!

In [1]:
# Import necessary libraries
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import numpy as np

# --- 1. Setup the Quantum Circuit ---
n_qubits = 6
# The bitstring we are looking for (the "marked" item)
marked_state = '101010' 

# Create a quantum circuit
qc = QuantumCircuit(n_qubits)

# --- 2. Create the Oracle ---
# The oracle flips the phase of the marked state
def oracle(qc, marked_state):
    """Applies the oracle for the given marked state."""
    # Reverse the marked_state string because Qiskit's qubit ordering is from right to left (q0, q1, ...)
    reversed_marked_state = marked_state[::-1]
    
    # Apply X gates to the qubits that are '0' in the marked state
    for i, bit in enumerate(reversed_marked_state):
        if bit == '0':
            qc.x(i)
            
    # Apply a multi-controlled Z gate (MCZ)
    # This flips the phase of the state |111...1>
    qc.h(n_qubits - 1)
    qc.mct(list(range(n_qubits - 1)), n_qubits - 1)  # multi-controlled-toffoli
    qc.h(n_qubits - 1)
    
    # Apply X gates again to revert the qubits to their original state
    for i, bit in enumerate(reversed_marked_state):
        if bit == '0':
            qc.x(i)
            
    qc.barrier()


# --- 3. Create the Diffuser (Grover Operator) ---
def diffuser(qc, n_qubits):
    """Applies the Grover diffusion operator."""
    # Apply Hadamard gates to all qubits
    qc.h(range(n_qubits))
    
    # Apply X gates to all qubits
    qc.x(range(n_qubits))
    
    # Apply multi-controlled Z gate
    qc.h(n_qubits - 1)
    qc.mct(list(range(n_qubits - 1)), n_qubits - 1)
    qc.h(n_qubits - 1)
    
    # Apply X gates to all qubits
    qc.x(range(n_qubits))
    
    # Apply Hadamard gates to all qubits
    qc.h(range(n_qubits))
    
    qc.barrier()

# --- 4. Build the full Grover's Algorithm Circuit ---

# Start with a uniform superposition
qc.h(range(n_qubits))
qc.barrier()

# Determine the optimal number of iterations
# For N = 2^n, the optimal number of iterations is around (pi/4) * sqrt(N)
iterations = int(np.round((np.pi / 4) * np.sqrt(2**n_qubits)))
print(f"Number of qubits: {n_qubits}")
print(f"Marked state: {marked_state}")
print(f"Optimal number of Grover iterations: {iterations}")


# Apply the Oracle and Diffuser for the optimal number of times
for _ in range(iterations):
    oracle(qc, marked_state)
    diffuser(qc, n_qubits)

# --- 5. Measurement ---
# Measure all qubits
qc.measure_all()

# --- 6. Simulation ---
# Use the Aer simulator
backend = Aer.get_backend('qasm_simulator')
# Execute the circuit
job = execute(qc, backend, shots=1024)
result = job.result()
counts = result.get_counts(qc)

# --- 7. Display Results ---
print("\\nSimulation Results:")
print(counts)
# Plot the results
plot_histogram(counts)


ModuleNotFoundError: No module named 'qiskit'

In [2]:
%pip install qiskit

Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.
